upzip image and mask Data

In [0]:
#! unzip "train.zip"

image and mask path

In [0]:
from glob import glob
import os
import numpy as np
from matplotlib import pyplot as plt


path_to_train = 'train'
#os.chdir(path_to_train)
glob_train_imgs = os.path.join(path_to_train, '*_sat.jpg')
glob_train_masks = os.path.join(path_to_train, '*_msk.png')

train_img_paths = glob(glob_train_imgs)
train_mask_paths = glob(glob_train_masks)
print(train_img_paths[:10])
print(train_mask_paths[:10])

['train/44286_sat.jpg', 'train/25543_sat.jpg', 'train/49323_sat.jpg', 'train/8358_sat.jpg', 'train/50116_sat.jpg', 'train/50762_sat.jpg', 'train/35111_sat.jpg', 'train/4879_sat.jpg', 'train/43967_sat.jpg', 'train/21077_sat.jpg']
['train/7181_msk.png', 'train/5529_msk.png', 'train/19553_msk.png', 'train/41031_msk.png', 'train/50738_msk.png', 'train/46794_msk.png', 'train/37746_msk.png', 'train/37833_msk.png', 'train/9013_msk.png', 'train/38605_msk.png']


image generator from path

In [0]:
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray

# This will be useful so we can construct the corresponding mask
def get_img_id(img_path):
    img_basename = os.path.basename(img_path)
    img_id = os.path.splitext(img_basename)[0][:-len('_sat')]
    return img_id

# Write it like a normal function
def image_gen(img_paths, img_size=(256, 256)):
    # Iterate over all the image paths
    for img_path in img_paths:
        
        # Construct the corresponding mask path
        img_id = get_img_id(img_path)
        mask_path = os.path.join(path_to_train, img_id + '_msk.png')
        
        # Load the image and mask, and normalize it to 0-1 range
        img = imread(img_path) / 255.
        mask = rgb2gray(imread(mask_path))
        
        # Resize the images
        #img = resize(img, img_size, preserve_range=True)
        #mask = resize(mask, img_size, mode='constant', preserve_range=True)
        # Turn the mask back into a 0-1 mask
        mask = (mask >= 0.5).astype(float)
        mask = np.expand_dims(mask, axis=-1)
        yield img, mask

metrics for training and testing

In [0]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import tensorflow as tf


smooth = 1e-9

# This is the competition metric implemented using Keras


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * (K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

# We'll construct a Keras Loss that incorporates the DICE score
def dice_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1. - (2. * intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.)

def bce_dice_loss(y_true, y_pred):
    return  0.5*binary_crossentropy(y_true, y_pred) + 0.5*dice_loss(y_true, y_pred)
  
def more_loss(y_true, y_pred):
    return 2*dice_loss(y_true, y_pred)
  
def own_loss(y_true, y_pred):
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    #y_pred_f = K.flatten(y_pred)
    a = tf.cast(tf.constant(1),'float32')
    b = tf.reduce_sum(y_true)
    return tf.cond(b < a, lambda: more_loss(y_true, y_pred), lambda: dice_loss(y_true, y_pred))

image batch generator

In [0]:
import numpy as np
import random
#import img_msk_aug_gen as gen
# Keras takes its input in batches 
# (i.e. a batch size of 32 would correspond to 32 images and 32 masks from the generator)
# The generator should run forever
def image_batch_generator(img_paths, batchsize=32):
  #img_paths = np.random.shuffle(img_paths)
  while True:
    #shuffle image paths
    img_paths = random.sample(img_paths, len(img_paths))
    ig = image_gen(img_paths)
    batch_img, batch_mask = [], []

    for img, mask in ig:
        # Add the image and mask to the batch
        batch_img.append(img)
        batch_mask.append(mask)
        # If we've reached our batchsize, yield the batch and reset
        if len(batch_img) == batchsize:
            yield np.stack(batch_img, axis=0), np.stack(batch_mask, axis=0)
            batch_img, batch_mask = [], []

    # If we have an nonempty batch left, yield it out and reset
    if len(batch_img) != 0:
        yield np.stack(batch_img, axis=0), np.stack(batch_mask, axis=0)
        batch_img, batch_mask = [], []   

image augamentation batch generator

In [0]:
from skimage.transform import *
from skimage.exposure import adjust_gamma
operations = ['original', 'rot90','rot180','rot270', 'flipud', 'fliplr', 'bright','dark' ]
#angles = [90,180,270]
#axises = ['ud','lr']
#brights = ['bright','dark']
#translations = [(-50,0),(0,-50),(50,0),(0,50)]
def image_aug_batch_generator(img_paths, batchsize = 32):
    while True:
        img_paths = random.sample(img_paths, len(img_paths))
        ig = image_gen(img_paths)
        batch_image, batch_mask = [],[]
        for image, mask in ig:
            operation = np.random.choice(operations,1)
            if operation is 'rot90':
              image = rotate(image, 90)
              mask = rotate(mask, 90)
            elif operation is 'rot180':
              image = rotate(image, 180)
              mask = rotate(mask, 180)
            elif operation is 'rot270':
              image = rotate(image, 270)
              mask = rotate(mask, 270)
            elif operation is 'bright':
              image = adjust_gamma(image, 0.5)
              mask = mask
            elif operation is 'dark':
              image = adjust_gamma(image, 2)
              mask = mask
            elif operation is 'flipud':
              image = np.flipud(iamge)
              mask = np.flipud(mask)
            elif operation is 'fliplr':
              image = np.fliplr(image)
              mask = np.fliplr(mask)
            else:
              image = image
              mask = mask
            
            batch_image.append(image)
            batch_mask.append(mask)
            if len(batch_image) == batchsize:
                yield np.stack(batch_image, axis = 0), np.stack(batch_mask, axis = 0)
                batch_image, batch_mask = [],[]
        
        if len(batch_image)!=0:
            yield np.stack(batch_image, axis = 0), np.stack(batch_mask, axis = 0)
            batch_iamge, batch_mask = [],[]

In [0]:
#from skimage.exposure import *
#image = imread('68707_sat.jpg')
#image_adj_0 = adjust_gamma(image,0.5)
#image_adj_2 = adjust_gamma(image, 2)
#plt.imshow(image)
#plt.show()
#plt.imshow(image_adj_0)
#plt.show()
#plt.imshow(image_adj_2)
#plt.show()


model training

Saving models

In [0]:
train_paths = []
for train_img_path in train_img_paths:
  img_id = get_img_id(train_img_path)
  mask_path = os.path.join(path_to_train, img_id + '_msk.png')
  mask = rgb2gray(imread(mask_path))
  mask = (mask >= 0.5).astype(float)
  area = np.sum(mask)
  if area > 200:
    train_paths.append(train_img_path)

In [0]:
from sklearn.model_selection import train_test_split
#from unet_simple_crfrnn import *
from dlinknet import *
from keras.callbacks import *

BATCHSIZE = 8
print('train_paths length is %d' %(len(train_paths)))
# Split the data into a train and validation set
train_img_paths, val_img_paths = train_test_split(train_paths, test_size=0.15)

# Create the train and validation generators
traingen = image_aug_batch_generator(train_img_paths, batchsize=BATCHSIZE)
valgen = image_batch_generator(val_img_paths, batchsize=BATCHSIZE)


def calc_steps(data_len, batchsize):
    return (data_len + batchsize - 1) // batchsize

# Calculate the steps per epoch
train_steps = calc_steps(len(train_img_paths), BATCHSIZE)
val_steps = (calc_steps(len(val_img_paths), BATCHSIZE))

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=1e-6)
checkpointer = ModelCheckpoint('dlinknet.h5', verbose=1, save_best_only=True)
earlystop = EarlyStopping(monitor = 'val_loss', patience = 20)

#from keras.models import load_model
#model = load_model("unet_dilation_reg_4.h5", custom_objects = {"bce_dice_loss":bce_dice_loss,"dice_coef":dice_coef})
#print('successfully loading')
model = dlinknet_34()
model.load_weights('weights_dlinknet_20.h5')
#model.summary()
model.compile(Adam(lr=1e-4), loss = dice_loss, metrics=[dice_coef])
# Train the model
history = model.fit_generator(
    traingen, 
    steps_per_epoch=train_steps, 
    epochs=80, # Change this to a larger number to train for longer
    validation_data=valgen, 
    validation_steps=val_steps, 
    verbose=1,
    callbacks = [reduce_lr,checkpointer, earlystop],
    max_queue_size=2,# Change this number based on memory restrictions
)

train_paths length is 8900


OSError: ignored

In [0]:
model.save_weights('weights_dlinknet_100.h5')

In [0]:
# Split the data into a train and validation set
train_img_paths, val_img_paths = train_test_split(train_img_paths, test_size=0.15)

#to store low training score img paths, 
train_scores = {}
train_paths = []
with open('train_paths.txt', 'w') as f:
    for train_img_path in train_img_paths:
        img_id = get_img_id(train_img_path)
        ori_img = imread(train_img_path)
        test_img = ori_img/255
        test_img = np.expand_dims(test_img, axis = 0)
        mask_path = os.path.join(path_to_train, img_id + '_msk.png')
        mask = rgb2gray(imread(mask_path))
        mask = (mask >= 0.5).astype(float)
        mask = np.expand_dims(mask, axis = 0)
        mask = np.expand_dims(mask, axis = -1)
        loss, dice_coef = model.evaluate(test_img, mask)
        if dice_coef < 0.6:
            train_paths.append(train_img_path)
            f.writelines(train_img_path + '\n')
            train_scores[img_id] = dice_coef

In [0]:
model.save('dlinknet_light_110.h5')
model.save_weights('weights_dlinknet_170.h5')

In [0]:
#to see the training score distribution
with open('train_scores.txt','w') as f:
    for k, v in train_scores.items():
        if v < 0.5:
            f.writelines('id is {}, score is {} '.format(k, v) + '\n')

In [0]:
print(len(train_paths))

In [0]:
with open('val_paths.txt', 'w') as f:
    for val_img_path in val_img_paths:
        f.write('%s\n' % val_img_path)

In [0]:
scores = {}
for val_img_path in val_img_paths:
    img_id = get_img_id(val_img_path)
    ori_img = imread(val_img_path)
    test_img = ori_img/255
    test_img = np.expand_dims(test_img, axis = 0)
    mask_path = os.path.join(path_to_train, img_id + '_msk.png')
    mask = rgb2gray(imread(mask_path))
    mask = (mask >= 0.5).astype(float)
    mask = np.expand_dims(mask, axis = 0)
    mask = np.expand_dims(mask, axis = -1)
    loss, dice_coef = model.evaluate(test_img, mask)
    scores[img_id] = [loss, dice_coef]

In [0]:
ids = []
low_scores = []
with open('low_score_ids.txt', 'w') as f:
    for k, v in scores.items():
        if v[1] <= 0.5:
            f.writelines('id is {}, score is {} '.format(k, v) + '\n')
            low_scores.append(k)

In [0]:
print(len(low_scores))

In [0]:
areas = []
count = 0
for train_img_path in train_img_paths:
  img = imread(train_img_path)
  img_id = get_img_id(train_img_path)
  mask_path = os.path.join(path_to_train, img_id + '_msk.png')
  mask = rgb2gray(imread(mask_path))
  area = np.sum(mask)
  if area > 0 and area < 100:
    areas.append(area)
    #print(img_id)
    count += 1

plt.hist(areas, bins = 100)
plt.show()
print(count)